In [ ]:
# 기본 라이브러리 및 경로 관련 모듈 임포트
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
import re

# 노트북에서 예쁜 표 출력을 위한 IPython display 함수
from IPython.display import display, HTML

# pandas 출력 옵션 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)


def print_table(df: pd.DataFrame, title: str | None = None, max_rows: int = 50, highlight_today: bool = False) -> None:
    """
    DataFrame을 출력하기 위한 helper function.
    - title이 있으면 제목을 HTML로 표시
    - 앞에서부터 최대 max_rows 행까지만 출력
    - highlight_today가 True이면 '수강 일자' 컬럼에 '★'가 포함된 행을 노란색으로 강조
    """
    if df is None or df.empty:
        print(f"⚠️ {title or '테이블'} - 표시할 데이터가 없습니다.")
        return

    # 제목이 있으면 HTML로 표시
    if title:
        display(HTML(f"<h3 style='color: #2c3e50; margin-bottom: 10px;'>{title}</h3>"))

    # 출력할 데이터 (최대 max_rows 행)
    display_df = df.head(max_rows).copy()

    # 오늘 날짜 강조가 필요한 경우 스타일 적용
    if highlight_today and '수강 일자' in display_df.columns:
        def highlight_today_row(row):
            """오늘 날짜가 포함된 행을 노란색으로 강조"""
            if '★' in str(row.get('수강 일자', '')):
                return ['background-color: #ffeb3b; font-weight: bold;'] * len(row)
            return [''] * len(row)
        
        styled_df = display_df.style.apply(highlight_today_row, axis=1)
        display(styled_df)
    else:
        # 일반 출력 (pandas의 기본 HTML 렌더링 사용)
        display(display_df)
    
    # 전체 행 수가 max_rows보다 많으면 안내 메시지
    if len(df) > max_rows:
        print(f"\n... (총 {len(df)}개 행 중 {max_rows}개만 표시)")


def time_to_seconds(time_str: str | float | int | None) -> int:
    """시간 문자열을 초 단위 정수로 변환하는 함수.

    - 지원 형식 예:
      - "0:10:52"  ->  10분 52초  (시:분:초)
      - "12:34"    ->  12분 34초  (분:초)
    """
    if pd.isna(time_str) or time_str == "":
        return 0
    try:
        parts = str(time_str).split(":")
        if len(parts) == 3:  # 시:분:초
            hours, minutes, seconds = map(int, parts)
            return hours * 3600 + minutes * 60 + seconds
        elif len(parts) == 2:  # 분:초
            minutes, seconds = map(int, parts)
            return minutes * 60 + seconds
        else:  # 인식할 수 없는 형식
            return 0
    except Exception:
        # 형식이 완전히 잘못된 경우 0초로 처리
        return 0


def seconds_to_time(total_seconds: int) -> str:
    """초 단위 정수를 "시:분:초" 형식의 문자열로 변환하는 함수."""
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    return f"{int(hours)}:{int(minutes):02d}:{int(seconds):02d}"


def parse_korean_date(date_str: str | None) -> datetime | None:
    """한국어 날짜 문자열을 datetime 객체로 변환하는 함수.

    예:
    - "12월 22일 (월)" -> 2025-12-22
    - "01월 02일 (금)" -> 2026-01-02

    규칙:
    - 12월  : 2025년
    - 그 외 : 2026년
    """
    if pd.isna(date_str) or date_str == "":
        return None

    try:
        # "12월 22일 (월)" 형식에서 월과 일 추출
        match = re.search(r"(\d+)월\s*(\d+)일", str(date_str))
        if match:
            month = int(match.group(1))
            day = int(match.group(2))

            # 12월은 2025년, 나머지는 2026년으로 매핑
            year = 2025 if month == 12 else 2026

            return datetime(year, month, day)
    except Exception:
        pass
    return None


def format_date_with_year(date_str: str | None) -> str | None:
    """날짜 문자열 앞에 연도를 붙여주는 함수.

    예:
    - "12월 22일 (월)" -> "2025년 12월 22일 (월)"
    """
    if pd.isna(date_str) or date_str == "":
        return date_str

    date_obj = parse_korean_date(date_str)
    if date_obj:
        # 원래 형식 유지하면서 연도 추가
        return f"{date_obj.year}년 {date_str}"
    return date_str

In [10]:
def load_data(data_path: str | Path | None = None) -> pd.DataFrame:
    """CSV 파일을 읽어서 DataFrame으로 반환하는 함수 (노트북 버전).

    - data_path가 None이면, 현재 노트북이 있는 디렉터리를 기준으로
      `asset/schedule.csv` 를 자동으로 찾습니다.
    - 첫 번째 줄(제목 행)은 건너뛰고 둘째 줄을 헤더로 사용합니다.
    """
    if data_path is None:
        # 현재 작업 디렉터리(노트북 위치)를 기준으로 경로 설정
        base_dir = Path.cwd()
        data_path = base_dir / "asset" / "schedule.csv"
    else:
        data_path = Path(data_path)

    # CSV 파일 읽기 (첫 번째 줄은 제목이므로 건너뛰기)
    df = pd.read_csv(data_path, skiprows=1)

    # ❗데이터는 하나도 삭제하지 않고, NaN만 보기 좋게 치환
    # - 전체 NaN을 빈 문자열("")로 바꾸면 표에서 NaN이 보이지 않습니다.
    df = df.fillna("")

    return df


def analyze_clip_time(df: pd.DataFrame) -> pd.DataFrame:
    """클립 시간 관련 통계를 계산하고 출력하는 함수.

    - 전체 클립 개수 / 총 시간 / 평균 시간
    - 강의별 클립 시간 요약
    - 파트별 클립 시간 요약
    """
    # 클립 시간 데이터 처리
    # 빈 문자열을 NaN으로 다시 변환 (계산을 위해)
    df_time = df.copy()
    df_time["클립 시간"] = df_time["클립 시간"].replace("", np.nan)

    # 클립 시간을 초 단위로 변환
    df_time["클립 시간(초)"] = df_time["클립 시간"].apply(time_to_seconds)

    # 전체 클립 시간 합산
    total_seconds = df_time["클립 시간(초)"].sum()
    total_time_str = seconds_to_time(total_seconds)

    print("=" * 60)
    print("📊 클립 시간 종합 분석")
    print("=" * 60)
    print(f"\n✅ 전체 클립 개수: {len(df_time)}개")
    print(f"✅ 클립 시간이 있는 데이터: {df_time['클립 시간(초)'].notna().sum()}개")
    print(f"✅ 총 클립 시간: {total_time_str} ({total_seconds:,}초)")
    print(f"✅ 평균 클립 시간: {seconds_to_time(int(df_time['클립 시간(초)'].mean()))}")

    # 강의별 클립 시간 합산
    if "강의명" in df_time.columns:
        lecture_summary = (
            df_time.groupby("강의명")["클립 시간(초)"].agg(["sum", "count", "mean"]).reset_index()
        )
        lecture_summary.columns = ["강의명", "총 시간(초)", "클립 개수", "평균 시간(초)"]
        lecture_summary["총 시간"] = lecture_summary["총 시간(초)"].apply(seconds_to_time)
        lecture_summary["평균 시간"] = lecture_summary["평균 시간(초)"].apply(
            lambda x: seconds_to_time(int(x)) if pd.notna(x) else "0:00:00"
        )

        print("\n📚 강의별 클립 시간 요약:")
        print("-" * 60)
        print_table(
            lecture_summary[["강의명", "클립 개수", "총 시간", "평균 시간"]],
            title="강의별 클립 시간 요약",
        )

    # 파트별 클립 시간 합산
    if "파트명" in df_time.columns:
        part_summary = (
            df_time.groupby("파트명")["클립 시간(초)"].agg(["sum", "count"]).reset_index()
        )
        part_summary.columns = ["파트명", "총 시간(초)", "클립 개수"]
        part_summary["총 시간"] = part_summary["총 시간(초)"].apply(seconds_to_time)
        part_summary = part_summary.sort_values("총 시간(초)", ascending=False)

        print("\n📖 파트별 클립 시간 요약:")
        print("-" * 60)
        print_table(
            part_summary[["파트명", "클립 개수", "총 시간"]].head(10),
            title="파트별 클립 시간 요약 (상위 10개)",
        )

    return df_time


def analyze_by_date(df_time: pd.DataFrame) -> None:
    """수강 일자(월/일) 기준으로 클립 시간을 집계하는 함수.

    - 일자별 클립 시간 합계 vs 일자별 수강 시간 합계
    - 일자별 차이(초)와 비고(어느 쪽이 더 큰지) 출력
    """
    # 수강 일자가 있는 데이터만 필터링
    df_date = df_time[df_time["수강 일자"] != ""].copy()

    if len(df_date) > 0:
        # 일자별 수강 시간도 초로 변환
        df_date["일자별 수강 시간(초)"] = df_date["일자별 수강 시간"].apply(time_to_seconds)

        date_summary_with_daily = df_date.groupby("수강 일자").agg(
            {
                "클립 시간(초)": "sum",
                "일자별 수강 시간(초)": "first",  # 같은 날짜면 같은 값이므로 first 사용
                "클립명": "count",
            }
        ).reset_index()

        date_summary_with_daily.columns = [
            "수강 일자",
            "클립 시간 합계(초)",
            "일자별 수강 시간(초)",
            "클립 개수",
        ]
        date_summary_with_daily["클립 시간 합계"] = date_summary_with_daily[
            "클립 시간 합계(초)"
        ].apply(seconds_to_time)
        date_summary_with_daily["일자별 수강 시간"] = date_summary_with_daily[
            "일자별 수강 시간(초)"
        ].apply(seconds_to_time)

        print("=" * 80)
        print("📅 수강 일자별 클립 시간 종합 정리")
        print("=" * 80)
        print(f"\n✅ 수강 일자가 기록된 데이터: {len(df_date)}개")
        print(f"✅ 수강 일자 수: {len(date_summary_with_daily)}일")

        # 전체 합계
        total_clip_seconds = date_summary_with_daily["클립 시간 합계(초)"].sum()
        total_daily_seconds = date_summary_with_daily["일자별 수강 시간(초)"].sum()

        print("\n📊 전체 요약:")
        print(f"   - 클립 시간 총합: {seconds_to_time(int(total_clip_seconds))}")
        print(f"   - 일자별 수강 시간 총합: {seconds_to_time(int(total_daily_seconds))}")

        print("\n📅 일자별 상세 내역:")
        print("-" * 80)
        detail_df = date_summary_with_daily[
            ["수강 일자", "클립 개수", "클립 시간 합계", "일자별 수강 시간"]
        ].sort_values("수강 일자")
        print_table(detail_df, title="수강 일자별 상세 내역")

        # 일자별 수강 시간과 클립 시간 합계 비교
        print("\n📈 일자별 수강 시간 vs 클립 시간 합계 비교:")
        print("-" * 80)
        comparison = date_summary_with_daily.copy()
        comparison["차이(초)"] = (
            comparison["일자별 수강 시간(초)"] - comparison["클립 시간 합계(초)"]
        )
        comparison["차이"] = comparison["차이(초)"].apply(
            lambda x: seconds_to_time(int(abs(x))) if pd.notna(x) else ""
        )
        comparison["비고"] = comparison.apply(
            lambda row: "일자별 시간이 더 많음"
            if row["차이(초)"] > 0
            else "클립 시간 합계가 더 많음"
            if row["차이(초)"] < 0
            else "동일",
            axis=1,
        )
        comp_df = comparison[
            ["수강 일자", "클립 시간 합계", "일자별 수강 시간", "차이", "비고"]
        ].sort_values("수강 일자")
        print_table(comp_df, title="일자별 수강 시간 vs 클립 시간 합계 비교")
    else:
        print("⚠️ 수강 일자가 기록된 데이터가 없습니다.")


def analyze_by_date_with_year(df_time: pd.DataFrame) -> None:
    """수강 일자에 연도를 붙이고, 오늘 날짜와 겹치는 행을 강조해서 보여주는 함수.

    - "12월 xx일"  -> 2025년
    - 그 외(01~11월) -> 2026년
    - 오늘 날짜와 같은 날짜는 "★" 마커로 강조
    """
    # 일자별 수강 시간을 초로 변환 (df_time에 추가)
    df_time["일자별 수강 시간"] = df_time["일자별 수강 시간"].replace("", np.nan)
    df_time["일자별 수강 시간(초)"] = df_time["일자별 수강 시간"].apply(time_to_seconds)

    # 수강일자에 연도 추가
    df_time["수강 일자_원본"] = df_time["수강 일자"].copy()
    df_time["수강 일자_날짜객체"] = df_time["수강 일자"].apply(parse_korean_date)
    df_time["수강 일자_연도포함"] = df_time["수강 일자"].apply(format_date_with_year)

    # 오늘 날짜
    today = datetime.now().date()
    print(f"📅 오늘 날짜: {today.strftime('%Y년 %m월 %d일')}")

    # 오늘 날짜와 일치하는 행 확인
    df_time["오늘_일치"] = df_time["수강 일자_날짜객체"].apply(
        lambda x: x.date() == today if x is not None else False
    )

    # 오늘 날짜와 일치하는 행 개수
    today_count = df_time["오늘_일치"].sum()
    print(f"✅ 오늘 날짜와 일치하는 수강 기록: {today_count}개\n")

    # 수강일자별 클립 시간 재조회 (연도 포함)
    df_date_updated = df_time[df_time["수강 일자_날짜객체"].notna()].copy()

    if len(df_date_updated) > 0:
        # 수강 일자별로 클립 시간 합산 (연도 포함된 날짜로 그룹화)
        date_summary_updated = df_date_updated.groupby("수강 일자_연도포함").agg(
            {
                "클립 시간(초)": "sum",
                "일자별 수강 시간(초)": "first",
                "클립명": "count",
                "수강 일자_날짜객체": "first",
                "오늘_일치": "any",
            }
        ).reset_index()

        date_summary_updated.columns = [
            "수강 일자",
            "클립 시간 합계(초)",
            "일자별 수강 시간(초)",
            "클립 개수",
            "날짜객체",
            "오늘_일치",
        ]
        date_summary_updated["클립 시간 합계"] = date_summary_updated[
            "클립 시간 합계(초)"
        ].apply(seconds_to_time)
        date_summary_updated["일자별 수강 시간"] = date_summary_updated[
            "일자별 수강 시간(초)"
        ].apply(seconds_to_time)

        # 날짜순으로 정렬
        date_summary_updated = date_summary_updated.sort_values("날짜객체")

        # 전체 합계
        total_clip_seconds = date_summary_updated["클립 시간 합계(초)"].sum()
        total_daily_seconds = date_summary_updated["일자별 수강 시간(초)"].sum()

        print("=" * 80)
        print("📅 수강 일자별 클립 시간 종합 정리 (연도 포함)")
        print("=" * 80)
        print(f"\n✅ 수강 일자가 기록된 데이터: {len(df_date_updated)}개")
        print(f"✅ 수강 일자 수: {len(date_summary_updated)}일")
        print("\n📊 전체 요약:")
        print(f"   - 클립 시간 총합: {seconds_to_time(int(total_clip_seconds))}")
        print(f"   - 일자별 수강 시간 총합: {seconds_to_time(int(total_daily_seconds))}")

        print("\n📅 일자별 상세 내역 (오늘 날짜는 ★로 표시):")
        print("-" * 80)

        # 표시할 컬럼만 선택
        display_df = date_summary_updated[
            ["수강 일자", "클립 개수", "클립 시간 합계", "일자별 수강 시간", "오늘_일치"]
        ].copy()

        # 오늘 날짜 표시를 위해 마커 추가
        display_df["수강 일자"] = display_df.apply(
            lambda row: f"★ {row['수강 일자']}" if row["오늘_일치"] else row["수강 일자"],
            axis=1,
        )

        # 오늘_일치 컬럼 제거하고 출력 (오늘 날짜 강조 활성화)
        print_table(
            display_df[["수강 일자", "클립 개수", "클립 시간 합계", "일자별 수강 시간"]],
            title="연도 포함 수강 일자별 상세 내역 (★ = 오늘)",
            highlight_today=True,  # 오늘 날짜 강조 활성화
        )

        # 오늘 날짜와 일치하는 행만 별도로 표시
        if today_count > 0:
            print(f"\n🎯 오늘 날짜({today.strftime('%Y년 %m월 %d일')}) 수강 기록:")
            print("-" * 80)
            today_records = date_summary_updated[date_summary_updated["오늘_일치"]][
                ["수강 일자", "클립 개수", "클립 시간 합계", "일자별 수강 시간"]
            ]
            print_table(today_records, title="오늘 날짜 수강 기록")
    else:
        print("⚠️ 수강 일자가 기록된 데이터가 없습니다.")

In [9]:
# === 실제 실행 셀 ===
# 1. CSV 데이터 로드
# 2. 클립 시간 분석
# 3. 수강 일자별 분석
# 4. 연도 포함 수강 일자별 분석 + 오늘 날짜 강조

# 1) 데이터 로드
df = load_data()  # 기본값(None)이면 현재 노트북 기준으로 asset/schedule.csv 사용

print("데이터 shape:", df.shape)
print("컬럼명:", df.columns.tolist())
print("\n첫 5개 행:")
print_table(df.head(), title="원본 데이터 (상위 5행)")

# 2) 클립 시간 분석
df_time = analyze_clip_time(df)

# 3) 수강 일자별 분석
analyze_by_date(df_time)

# 4) 연도 포함 수강 일자별 분석 + 오늘 날짜 강조
analyze_by_date_with_year(df_time)

데이터 shape: (827, 17)
컬럼명: ['Unnamed: 0', '원 코스명 / ID', '강의명', '클립 번호', '파트명', '챕터명', '클립명', '클립 시간', '클립 ID', '분류', '수강 일자', '일자별 수강 시간', '과제 배치', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']

첫 5개 행:


,Unnamed: 0,원 코스명 / ID,강의명,클립 번호,파트명,챕터명,클립명,클립 시간,클립 ID,분류,수강 일자,일자별 수강 시간,과제 배치,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,,실패 없이 완주하는 파이썬 데이터 분석 입문 (212190),실패 없이 완주하는 파이썬 데이터 분석 입문,1,Part1 파이썬 기본기 다지기,Ch01. 파이썬에 대한 모든 것,Intro. Selena 강좌를 듣기 전 필독! 학습 방법 5단계,0:10:52,,필수,12월 22일 (월),1:15:15,,,,,
1,,실패 없이 완주하는 파이썬 데이터 분석 입문 (212190),실패 없이 완주하는 파이썬 데이터 분석 입문,2,Part1 파이썬 기본기 다지기,Ch01. 파이썬에 대한 모든 것,CH01-01. 파이썬 활용분야,0:09:44,,필수,,,,,,,
2,,실패 없이 완주하는 파이썬 데이터 분석 입문 (212190),실패 없이 완주하는 파이썬 데이터 분석 입문,3,Part1 파이썬 기본기 다지기,Ch02. 변수와 자료형,"CH02-01. 변수(variable), 자료형(data type), 숫자형(num...",0:13:50,,필수,,,,,"5538476:20251222,5538482:20251223,5538488:2025...",,
3,,실패 없이 완주하는 파이썬 데이터 분석 입문 (212190),실패 없이 완주하는 파이썬 데이터 분석 입문,4,Part1 파이썬 기본기 다지기,Ch02. 변수와 자료형,CH02-02. 문자열 자료형(string) I,0:13:48,,필수,,,,,"5538476:20251222,5538482:20251223,5538488:2025...",,
4,,실패 없이 완주하는 파이썬 데이터 분석 입문 (212190),실패 없이 완주하는 파이썬 데이터 분석 입문,5,Part1 파이썬 기본기 다지기,Ch02. 변수와 자료형,CH02-03. 문자열 자료형(string) II,0:10:21,,필수,,,,,,,


📊 클립 시간 종합 분석

✅ 전체 클립 개수: 827개
✅ 클립 시간이 있는 데이터: 827개
✅ 총 클립 시간: 234:17:41 (843,461초)
✅ 평균 클립 시간: 0:16:59

📚 강의별 클립 시간 요약:
------------------------------------------------------------


,강의명,클립 개수,총 시간,평균 시간
0,50개 프로젝트로 끝내는 머신러닝 Signature,18,6:09:40,0:20:32
1,RAG를 활용한 완성도 높은 LLM 서비스 구축 With langchain & ll...,29,8:56:36,0:18:30
2,[업스테이지] AI 기초 학습 : 딥러닝,40,16:24:23,0:24:36
3,[업스테이지] AI 실전 학습,76,31:51:49,0:25:09
4,[업스테이지] AI 실전 학습 : Information Retrieval 도메인 학습,27,5:28:50,0:12:10
5,[업스테이지] AI 심화 학습 : AI Data,31,9:06:38,0:17:38
6,[업스테이지] AI 심화 학습 : Natural Language Processing...,83,24:29:50,0:17:42
7,[업스테이지] AI 심화 학습 : 머신러닝,49,21:41:02,0:26:33
8,[업스테이지] AI 심화학습 : Computer Vision 도메인 학습,77,25:49:42,0:20:07
9,네카라쿠배 취업 끝장내는 파이썬 코딩테스트 마스터,157,22:01:12,0:08:24



📖 파트별 클립 시간 요약:
------------------------------------------------------------


...

,수강 일자,클립 시간 합계,일자별 수강 시간,차이,비고
0,01월 02일 (금),0:13:34,1:56:41,1:43:07,일자별 시간이 더 많음
1,01월 05일 (월),0:08:52,1:40:05,1:31:13,일자별 시간이 더 많음
2,01월 06일 (화),0:05:38,1:34:09,1:28:31,일자별 시간이 더 많음
3,01월 07일 (수),0:24:05,1:15:52,0:51:47,일자별 시간이 더 많음
4,01월 08일 (목),0:15:37,1:09:57,0:54:20,일자별 시간이 더 많음
5,01월 09일 (금),0:14:57,1:02:52,0:47:55,일자별 시간이 더 많음
6,01월 12일 (월),0:17:24,1:12:55,0:55:31,일자별 시간이 더 많음
7,01월 13일 (화),0:25:17,3:36:13,3:10:56,일자별 시간이 더 많음
8,01월 14일 (수),0:38:32,3:45:28,3:06:56,일자별 시간이 더 많음
9,01월 15일 (목),0:07:39,2:44:28,2:36:49,일자별 시간이 더 많음



... (총 152개 행 중 50개만 표시)
📅 오늘 날짜: 2026년 01월 14일
✅ 오늘 날짜와 일치하는 수강 기록: 1개

📅 수강 일자별 클립 시간 종합 정리 (연도 포함)

✅ 수강 일자가 기록된 데이터: 152개
✅ 수강 일자 수: 152일

📊 전체 요약:
   - 클립 시간 총합: 43:57:26
   - 일자별 수강 시간 총합: 235:31:58

📅 일자별 상세 내역 (오늘 날짜는 ★로 표시):
--------------------------------------------------------------------------------


,수강 일자,클립 개수,클립 시간 합계,일자별 수강 시간
0,2025년 12월 22일 (월),1,0:10:52,1:15:15
1,2025년 12월 23일 (화),1,0:13:22,1:24:41
2,2025년 12월 24일 (수),1,0:17:12,1:14:30
3,2025년 12월 26일 (금),1,0:13:20,1:29:15
4,2025년 12월 29일 (월),1,0:10:16,0:52:33
5,2025년 12월 30일 (화),1,0:14:00,1:20:13
6,2025년 12월 31일 (수),1,0:11:47,1:30:30
7,2026년 01월 02일 (금),1,0:13:34,1:56:41
8,2026년 01월 05일 (월),1,0:08:52,1:40:05
9,2026년 01월 06일 (화),1,0:05:38,1:34:09



... (총 152개 행 중 50개만 표시)

🎯 오늘 날짜(2026년 01월 14일) 수강 기록:
--------------------------------------------------------------------------------


,수강 일자,클립 개수,클립 시간 합계,일자별 수강 시간
15,2026년 01월 14일 (수),1,0:38:32,3:45:28
